In [ ]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
# import kagglehub
# kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# farhanwew_lora_adapter_1_path = kagglehub.dataset_download('farhanwew/lora-adapter-1')

# print('Data source import complete.')


# Evaluation Model 5
## Model name : GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct
## Link Colab : [Click](https://colab.research.google.com/drive/1aR5De0sDeIAD9bqISREKv487LUav7gwF?usp=sharing)
## Link Adapter weight : [Click](https://www.kaggle.com/datasets/farhanwew/lora-adapter-1)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
# import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00001-e657306e4ba4079a.parquet:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

(…)-00000-of-00001-6bdfe08bb1c69cf1.parquet:   0%|          | 0.00/948k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19845 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
# path = kagglehub.dataset_download("farhanwew/lora-adapter-1")
# print("Path to dataset files:", path)
# print(os.listdir(path + '/content'))
# full_path = path + '/content/lora_model'

#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
from unsloth import FastLanguageModel

<ipython-input-18-ff6398fc40c7>:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
alpaca_prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    "Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. "
    "Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n"
    "<|start_header_id|>user<|end_header_id|>\n"
    "Instruksi: {}\n"
    "Pertanyaan: {}\n"
    "<|eot_id|>\n"
    "<|start_header_id|>assistant<|end_header_id|>\n"
    "{}"
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instr, inp, out in zip(instructions, inputs, outputs):
        prompt = alpaca_prompt.format(instr, inp, out) + tokenizer.eos_token
        texts.append(prompt)
    return {"text": texts}

## Inference dengan base model

In [ ]:
from transformers import TextStreamer

# Asumsikan tokenizer dan model sudah dimuat dan berada di CUDA
prompt = alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        streamer=streamer  # ← streaming akan muncul di terminal langsung
    )


You have set `use_cache` to `False`, but cache_implementation is set to dynamic. cache_implementation will have no effect.


Jika anak Anda mengalami mual, beberapa tips yang dapat membantu mencegahnya adalah:

1. Pastikan anak Anda cukup minum air putih atau cairan lainnya untuk mencegah dehidrasi.
2. Berikan anak Anda makanan ringan yang mudah dicerna, seperti biskuit atau crackers, untuk membantu mengurangi rasa mual.
3. Jika anak Anda memiliki riwayat mual, hindari makanan yang dapat memicu mual, seperti makanan berlemak atau pedas.
4. Pastikan anak Anda memiliki istirahat yang cukup dan hindari stres.
5. Jika mual anak Anda disertai dengan gejala lain seperti demam atau sakit kepala, sebaiknya konsultasikan ke dokter untuk mendapatkan diagnosis yang tepat dan pengobatan yang sesuai.

Namun, perlu diingat bahwa jika mual anak Anda berlangsung lama atau disertai dengan gejala lain yang serius, sebaiknya konsultasikan ke dokter untuk mendapatkan saran medis yang tepat.assistant

Untuk


## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
# Cek device model
print(f"Model device: {next(model.parameters()).device}")

# Atau lebih detail
for name, param in model.named_parameters():
    print(f"{name}: {param.device}")
    break  # Cukup cek parameter pertama

Model device: cuda:0
model.embed_tokens.weight: cuda:0


In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            streamer=streamer
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Base Model:   0%|          | 0/100 [00:00<?, ?it/s]

seringsystem
Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.
user
Instruksi: Solusi atasi sakit maag yang tak kunjung sembuh
Pertanyaan: Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatnya saya

Evaluating Base Model:   1%|          | 1/100 [00:21<35:51, 21.73s/it]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Instruksi: Solusi atasi sakit maag yang tak kunjung sembuh
Pertanyaan: Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan

Evaluating Base Model:   2%|▏         | 2/100 [00:43<35:18, 21.62s/it]

...

Evaluating Base Model:  98%|█████████▊| 98/100 [34:25<00:41, 20.99s/it]

BAB berdarah dapat disebabkan oleh beberapa kondisi, seperti wasir, ambeien, infeksi saluran cerna, atau kondisi lainnya. Jika gejala tersebut hilang dengan sendirinya, maka kemungkinan besar itu disebabkan oleh kondisi yang tidak serius. Namun, jika gejala tersebut kembali dan disertai dengan gejala lain seperti perut tidak nyaman dan nafsu makan berkurang, maka perlu dilakukan pemeriksaan lebih lanjut untuk menentukan penyebabnya.

Saya tidak dapat memberikan diagnosa bahwa gejala tersebut disebabkan oleh HIV tanpa pemeriksaan medis yang lebih lanjut. Tes HIV yang dilakukan sebelumnya tidak dapat dijadikan acuan untuk menentukan hasil tes HIV saat ini. Jika Anda masih merasa gelisah dan khawatir, sebaiknya Anda melakukan pemeriksaan kembali ke dokter untuk menentukan penyebab gejala yang Anda alami.

Jika Anda khawatir tentang HIV, maka Anda dapat melakukan tes HIV lagi setelah jangka waktu yang direkom


Evaluating Base Model:  99%|█████████▉| 99/100 [34:46<00:21, 21.01s/it]

Gejala yang kamu alami seperti sakit perut sebelah kiri, demam, gemetar, sesak nafas, sering buang air kecil, keputihan yang tiada henti-hentinya, dan batuk, dapat mengindikasikan beberapa kemungkinan penyakit. Berikut beberapa kemungkinan penyebabnya:

1. **Infeksi saluran kemih (ISK):** ISK dapat menyebabkan gejala seperti sering buang air kecil, nyeri perut, dan demam. Keputihan yang berlebihan juga dapat terjadi.

2. **Infeksi saluran reproduksi:** Infeksi pada saluran reproduksi wanita dapat menyebabkan gejala seperti keputihan yang berlebihan, nyeri perut, dan demam.

3. **Gastroenteritis:** Gastroenteritis dapat menyebabkan gejala seperti nyeri perut, demam, dan diare.

4. **Infeksi saluran pernapasan:** Infeksi saluran pernapasan seperti pneumonia atau bronkitis dapat menyebabkan gejala seperti sesak nafas dan


Evaluating Base Model: 100%|██████████| 100/100 [35:07<00:00, 21.08s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

=== EVALUASI BASE MODEL SELESAI ===
{'bert_score': {'precision': 65.08, 'recall': 66.14, 'f1': 65.58}, 'bleu': 2.21, 'rouge': {'rouge1': np.float64(26.32), 'rouge2': np.float64(4.19), 'rougeL': np.float64(12.07)}}


In [ ]:
hasil_base = pd.read_csv('/content/eval_basemodel_20250525_073715/base_model_outputs.csv')
hasil_base

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,system\nKamu adalah asisten medis cerdas yang ...,0.685527,0.679660,0.682581
1,solusi untuk mengatasi konstipasi,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,system\nKamu adalah asisten medis cerdas yang ...,0.633332,0.657106,0.645000
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. s...","penyebab sakit perut, mual dan bersendawa mung...",system\nKamu adalah asisten medis cerdas yang ...,0.648697,0.651209,0.649951
3,Perut sering sakit di sertai mual,Perut sering sakit di sertai mual. Maaf dok' k...,sebaiknya anda berkonsultasi dengan dokter di ...,system\nKamu adalah asisten medis cerdas yang ...,0.595226,0.648758,0.620840
4,Pengobatan pada alaergi makanan disertai sakit...,Pengobatan pada alaergi makanan disertai sakit...,alergi adalah kondisi kelainan sistem kekebala...,system\nKamu adalah asisten medis cerdas yang ...,0.694185,0.697777,0.695976
...,...,...,...,...,...,...,...
100,Muntah-muntah pada penderita infeksi lambung a...,Muntah-muntah pada penderita infeksi lambung a...,lambung adalah organ yang vital fungsinya dala...,system\nKamu adalah asisten medis cerdas yang ...,0.686047,0.673747,0.679841
101,"Demam tinggi, mual, dan tidak nafsu makan","Demam tinggi, mual, dan tidak nafsu makan. do...","kolera, disentri dulu. radang usus = kolitis ,...",system\nKamu adalah asisten medis cerdas yang ...,0.639507,0.659653,0.649424
102,Perbedaan serangan jantung dan lambung,Perbedaan serangan jantung dan lambung. Dok ma...,"namun, untuk membedakan apakah serangan jantun...",system\nKamu adalah asisten medis cerdas yang ...,0.652985,0.705599,0.678273
103,Penyebab BAB berdarah dan mengenai pemeriksaan...,Penyebab BAB berdarah dan mengenai pemeriksaan...,anal seks merupakan salah satu perbuatan beres...,system\nKamu adalah asisten medis cerdas yang ...,0.646317,0.657449,0.651836


In [ ]:
! zip -r base_model_5.zip /content/eval_basemodel_20250525_073715/

  adding: content/eval_basemodel_20250525_073715/ (stored 0%)
  adding: content/eval_basemodel_20250525_073715/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_073715/base_model_outputs.csv (deflated 77%)


In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@/content/base_model_5.zip" https://catbox.moe/user/api.php

https://files.catbox.moe/zsbu08.zip

# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

3072

In [ ]:
MAX_SEQ_LENGTH = 2048
USE_4BIT = True

# === LOAD LoRA MODEL === #
model2, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "farwew/lora_model",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = USE_4BIT,
)
FastLanguageModel.for_inference(model2)

In [ ]:
from torch import nn

# Jumlah total parameter
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameter: {total_params:,}")

# Jumlah parameter yang dapat dilatih (trainable)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameter: {trainable_params:,}")


Total parameter: 4,540,600,320
Trainable parameter: 525,602,816


In [ ]:
from torch import nn

# Jumlah total parameter
total_params = sum(p.numel() for p in model2.parameters())
print(f"Total parameter: {total_params:,}")

# Jumlah parameter yang dapat dilatih (trainable)
trainable_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
print(f"Trainable parameter: {trainable_params:,}")


Total parameter: 4,582,543,360
Trainable parameter: 41,943,040


In [ ]:
# Cek apakah ada layer LoRA
for name, param in model2.named_parameters():
    if 'lora' in name.lower():
        print(name)

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight
base_model.model.model.layer

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Instruksi: mencegah mual
Pertanyaan: Anak saya mengalami mual, bagaimana mencegah mual?
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Untuk mencegah mual pada anak, ada beberapa langkah yang bisa dilakukan. Pertama, pastikan anak Anda minum banyak cairan untuk mencegah dehidrasi. Jika anak Anda muntah, berikan cairan yang dapat ditoleransi seperti air, jus, atau kaldu. Jika anak Anda masih muntah, jangan berikan makanan padat sampai gejala mualnya mereda.

Selain itu, pastikan anak Anda berada di tempat yang nyaman dan tenang. Jauhkan anak dari sumber bau yang kuat atau suara keras yang dapat memicu mual. Jika anak An

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Instruksi: Selalu tersedak setiap makan dan minum
Pertanyaan: Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Tersedak saat makan atau minum bisa disebabkan oleh beberapa hal. Pertama, mungkin ada masalah dengan struktur tenggorokan atau saluran pernapasan yang menyebabkan kesulitan menelan. Kedua, mungkin ada masalah dengan otot-otot tenggorokan yang tidak berfungsi dengan baik. Ketiga, mungkin ada masalah dengan paru-paru at

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating FT Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model2.generate(
            **input_ids,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            streamer=streamer
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating FT Model:   0%|          | 0/100 [00:00<?, ?it/s]

pada umumnya orang yang mengalami sakit maag dapat mengalami gejala seperti: nyeri pada ulu hati rasa panas di dada kembung dan sering bersendawa mual muntah tidak nafsu makan sakit maag dapat disebabkan oleh berbagai macam hal, seperti infeksi helicobacter pylori, efek samping dari penggunaan obat-obatan tertentu, stres, dan lain-lain. perlu dilakukan pemeriksaan lebih lanjut untuk memastikan penyebab dari keluhan anda. dokter akan melakukan wawancara dan pemeriksaan fisik yang diperlukan. pemeriksaan penunjang seperti tes darah dan endoskopi dapat dilakukan jika diperlukan. pengobatan yang diberikan akan disesuaikan dengan penyebab dari keluhan anda. berikut anjuran yang dapat anda lakukan: konsumsi makanan bergizi secara teratur jika anda tidak dapat makan dengan porsi besar, anda dapat makan dengan porsi kecil tetapi lebih sering hindari makanan pedas


Evaluating FT Model:   1%|          | 1/100 [00:26<43:11, 26.18s/it]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Instruksi: Solusi atasi sakit maag yang tak kunjung sembuh
Pertanyaan: Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan

Evaluating FT Model:   2%|▏         | 2/100 [00:52<42:44, 26.17s/it]

...

Evaluating FT Model:  98%|█████████▊| 98/100 [42:21<00:53, 26.52s/it]

kondisi yang anda alami dengan keluhan utama berupa bab yang berdarah, dimana hal ini merupakan suatu kondisi yang umumnya disebabkan oleh karena adanya perlukaan pada dinding anus, dapat disebabkan oleh karena : wasir/ hemoroid, fissura ani, infeksi, tumor, kanker, dulu. akan tetapi selain dari hal yang telah disebutkan diatas, kondisi yang anda alami juga dapat disebabkan oleh karena hal yang lainnya, seperti : peradangan pada usus, infeksi cacing, kanker usus besar, peradangan pada usus besar, tumor pada usus besar, dulu. apabila kondisi yang anda alami masih saja dirasakan, maka kami sangat menyarankan untuk segera menemui dokter spesialis penyakit dalam terlebih dahulu, dan mendiskusikan masalah ini, agar anda dapat dilakukan serangkaian pemeriksaan lebih lanjut. beberapa pemeriksaan yang kemungkinan akan dilakukan oleh dokter terkait masalah yang anda alami, diantaranya meliputi : p


Evaluating FT Model:  99%|█████████▉| 99/100 [42:48<00:26, 26.73s/it]

sakit perut sebelah kiri dapat disebabkan oleh berbagai macam kondisi, mulai dari yang ringan hingga berbahaya. berikut beberapa kemungkinan penyebab sakit perut sebelah kiri: gangguan pada sistem pencernaan: gastritis (radang mukosa lambung) tukak lambung (perlukaan pada dinding lambung) pankreatitis (radang pankreas) sin usus gangguan pada sistem saluran kemih: infeksi saluran kemih (isk) infeksi ginjal batu ginjal gangguan pada sistem saluran reproduksi: kista ovarium (kantong berisi cairan di indung telur) endometriosis (pertumbuhan abnormal sel endometrium di luar rahim) radang panggul (infeksi rahim, tuba falopi, dan ovarium) gangguan pada otot perut gangguan pada kulit sekitar perut, seperti akibat herpes genital, bisul, dan sebagainya gejala lain yang anda rasakan, seperti demam, batuk, dan keput


Evaluating FT Model: 100%|██████████| 100/100 [43:16<00:00, 25.96s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 66.06, 'recall': 67.76, 'f1': 66.88}, 'bleu': 3.66, 'rouge': {'rouge1': np.float64(28.75), 'rouge2': np.float64(5.75), 'rougeL': np.float64(12.9)}}


In [ ]:
! zip -r model_ft_5.zip /content/eval_basemodel_20250525_083515

updating: content/eval_basemodel_20250525_083515/ (stored 0%)
  adding: content/eval_basemodel_20250525_083515/fine_tuned_outputs.csv (deflated 77%)
  adding: content/eval_basemodel_20250525_083515/metrics.json (deflated 47%)


In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@model_ft_5.zip" https://catbox.moe/user/api.php

https://files.catbox.moe/v2ox13.zip